In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd


import time
import os.path

import shapely.geometry as geom
import geopandas as gpd
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from shapely.geometry import Point

## step 1 obtain OSM network

In [2]:
"Get the OSM Network"
network_type = 'auto'
import osm2gmns as og

net = og.getNetFromOSMFile('tmc.osm',network_type=(network_type), default_lanes=True, default_speed=True)

# og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=os.getcwd())

osm2gmns, version 0.4.2


## step 2 Convert tmc Data into node and link

In [2]:
link_base = pd.read_csv('link.csv', low_memory=False)
link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk'])]
link_base = link_base[-link_base['name'].isna()]
# link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk','primary','secondary'])]
link_base = link_base.reset_index()
link_base = link_base.drop(['index'], 1)

In [3]:
multiline_string_base_list = []
multiline_string_base_list_sub = []
for j in link_base.index:
    link_base_geometry_list = link_base.loc[j,'geometry'][12:-1].split(", ")
    for link_base_geometry in link_base_geometry_list:
        multiline_string_base_list_sub.append((float(link_base_geometry.split(" ")[0]),float(link_base_geometry.split(" ")[1])))
    multiline_string_base_list_sub = tuple(multiline_string_base_list_sub)
    multiline_string_base_list.append(multiline_string_base_list_sub)
    multiline_string_base_list_sub = []

from shapely.geometry import MultiLineString

line_base = MultiLineString(multiline_string_base_list) 

In [4]:
"Convert TMC Data into GMNS Format"
tmc = pd.read_csv('TMC_Identification.csv')
tmc = tmc.drop_duplicates(subset=['direction','road_order']).sort_values(by=['direction','road_order'])
tmc = tmc.reset_index()
tmc = tmc.drop(['index'], 1)
origin_tmc_num = len(tmc)

In [5]:
in_bbox_index_list = []
for i in tmc.index:
     if (tmc['start_longitude'][i] > line_base.bounds[0]) & (tmc['start_longitude'][i] < line_base.bounds[2]) & \
         (tmc['end_longitude'][i] > line_base.bounds[0]) & (tmc['end_longitude'][i] < line_base.bounds[2]) & \
             (tmc['start_latitude'][i] > line_base.bounds[1]) & (tmc['start_latitude'][i] < line_base.bounds[3]) & \
         (tmc['end_latitude'][i] > line_base.bounds[1]) & (tmc['end_latitude'][i] < line_base.bounds[3]):
         in_bbox_index_list.append(i)

tmc = tmc.loc[in_bbox_index_list]
tmc = tmc.reset_index()
tmc = tmc.drop(['index'], 1)
if len(in_bbox_index_list) < origin_tmc_num:
    print('base map cannot cover all TMC nodes,' + str(origin_tmc_num-len(in_bbox_index_list)) + 'tmc nodes are out of boundary box, please use larger base map')

In [6]:
tmc

,tmc,road,direction,intersection,state,county,zip,start_latitude,start_longitude,end_latitude,end_longitude,miles,road_order
0,110N05711,VA-7,EASTBOUND,VA-228/DRANESVILLE RD,VA,FAIRFAX COUNTY,20170,39.0144,-77.3715,39.0141,-77.3709,0.038620,38.0
1,110-05710,VA-7,EASTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0141,-77.3709,39.0127,-77.3684,0.164895,39.0
2,110N05710,VA-7,EASTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0127,-77.3684,39.0086,-77.3605,0.512158,40.0
3,110-05709,VA-7,EASTBOUND,SUGARLAND RD,VA,FAIRFAX COUNTY,20170,39.0086,-77.3605,39.0028,-77.3502,0.682589,41.0
4,110-05708,VA-7,EASTBOUND,VA-193/GEORGETOWN PIKE,VA,FAIRFAX COUNTY,20170,39.0028,-77.3502,39.0005,-77.3455,0.296731,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,110+05709,VA-7,WESTBOUND,SUGARLAND RD,VA,FAIRFAX COUNTY,20170,39.0010,-77.3460,39.0030,-77.3500,0.254799,45.0
58,110+05710,VA-7,WESTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0030,-77.3500,39.0090,-77.3609,0.718593,46.0
59,110P05710,VA-7,WESTBOUND,VA-7100/HOLLY KNOLL DR,VA,FAIRFAX COUNTY,20170,39.0090,-77.3609,39.0125,-77.3673,0.421999,47.0
60,110+05711,VA-7,WESTBOUND,VA-228/DRANESVILLE RD,VA,FAIRFAX COUNTY,20170,39.0125,-77.3673,39.0143,-77.3707,0.220479,48.0


In [7]:

'''build node.csv'''
print('converting tmc data into gmns format...')
p=1

node_tmc = pd.DataFrame()
node_tmc['name'] = None
node_tmc['x_coord'] = None
node_tmc['y_coord'] = None
node_tmc['z_coord'] = None
node_tmc['node_type'] = None
node_tmc['ctrl_type'] = None
node_tmc['zone_id'] = None
node_tmc['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
    else:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)

    if i > p/10 * len(tmc): 
        print(str(p*10)+"%"+' nodes completed!')
        p = p + 1

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude'], \
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'start_longitude'].astype(str) + " " + tmc.loc[i+1,'start_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc.index.name = 'node_id'

node_tmc.index += 100000001 #index from 0

node_tmc.to_csv('node_tmc.csv')
print('node_tmc.csv (' + str(len(node_tmc)) + ' nodes' + ') generated!')

converting tmc data into gmns format...
10% nodes completed!
20% nodes completed!
30% nodes completed!
40% nodes completed!
50% nodes completed!
60% nodes completed!
70% nodes completed!
80% nodes completed!
90% nodes completed!
node_tmc.csv (64 nodes) generated!


In [8]:
node_tmc

,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,geometry
node_id,,,,,,,,
100000001,110N05711,-77.3715,39.0144,None,tmc_start,None,None,POINT (-77.3715 39.0144)
100000002,110-05710,-77.3709,39.0141,None,tmc_start,None,None,POINT (-77.3709 39.0141)
100000003,110N05710,-77.3684,39.0127,None,tmc_start,None,None,POINT (-77.3684 39.0127)
100000004,110-05709,-77.3605,39.0086,None,tmc_start,None,None,POINT (-77.3605 39.0086)
100000005,110-05708,-77.3502,39.0028,None,tmc_start,None,None,POINT (-77.3502 39.0028)
...,...,...,...,...,...,...,...,...
100000060,110+05710,-77.3500,39.0030,None,tmc_start,None,None,POINT (-77.35 39.003)
100000061,110P05710,-77.3609,39.0090,None,tmc_start,None,None,POINT (-77.3609 39.009)
100000062,110+05711,-77.3673,39.0125,None,tmc_start,None,None,POINT (-77.3673 39.0125)


In [9]:
'''build link_tmc.csv'''
p = 1
link_tmc = pd.DataFrame()
link_tmc['name'] = None
link_tmc['corridor_id'] = None
link_tmc['corridor_link_order'] = None
link_tmc['from_node_id'] = None
link_tmc['to_node_id'] = None
link_tmc['directed'] = None
link_tmc['geometry_id'] = None
link_tmc['geometry'] = None
link_tmc['dir_flag'] = None
link_tmc['length'] = None
link_tmc['grade'] = None
link_tmc['facility_type'] = None
link_tmc['capacity'] = None
link_tmc['free_speed'] = None
link_tmc['lanes'] = None

for i in range(0,len(tmc)):
    link_tmc = link_tmc.append({'name': tmc.loc[i,'tmc'],\
                                'corridor_id': tmc.loc[i,'road']+'_'+tmc.loc[i,'direction'],\
                                'corridor_link_order' : tmc.loc[i,'road_order'],\
                                'from_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'start_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                'to_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'end_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                'directed': 1,\
                                'geometry_id': None,\
                                'geometry': "LINESTRING (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) + "," +\
                                    tmc.loc[i,'end_longitude'].astype(str) +" "+ tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                'dir_flag': 1,\
                                'length': tmc.loc[i,'miles'],\
                                'grade': None,\
                                'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                'capacity':None,\
                                'free_speed':None,\
                                'lanes': None}, ignore_index=True)

    if i > p/10 * len(tmc): 
        print(str(p*10)+"%"+' links completed!')
        p = p + 1
        
link_tmc.index.name = 'link_id'
link_tmc.index += 100000001


link_tmc.to_csv('link_tmc.csv')

print('link_tmc.csv (' + str(len(link_tmc)) + ' links' + ') generated!')

10% links completed!
20% links completed!
30% links completed!
40% links completed!
50% links completed!
60% links completed!
70% links completed!
80% links completed!
90% links completed!
link_tmc.csv (62 links) generated!


In [10]:
link_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,,
100000001,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,1,None,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",1,0.038620,None,None,None,None,None
100000002,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,1,None,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",1,0.164895,None,None,None,None,None
100000003,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,1,None,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",1,0.512158,None,None,None,None,None
100000004,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,1,None,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",1,0.682589,None,None,None,None,None
100000005,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,1,None,"LINESTRING (-77.3502 39.0028,-77.3455 39.0005)",1,0.296731,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000058,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,1,None,"LINESTRING (-77.346 39.001,-77.35 39.003)",1,0.254799,None,None,None,None,None
100000059,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,1,None,"LINESTRING (-77.35 39.003,-77.3609 39.009)",1,0.718593,None,None,None,None,None
100000060,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,1,None,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",1,0.421999,None,None,None,None,None


In [11]:
def volume_speed_func(speed,ffs=66,critical_density=35,mm=6): # fundamental diagram  (volume_delay fuction) 
    if speed > ffs:
        speed = ffs
    kernal=np.power(np.power(ffs/speed,mm),0.5)
    return speed*critical_density*np.power(kernal-1,1/mm)

In [12]:
## reading by detid
reading = pd.read_csv('Reading.csv')
reading = reading.loc[0:5000]


reading_dict = {}
gp = reading.groupby('tmc_code')
for key, form in gp:
    reading_dict[key] = {
        'measurement_tstamp':form['measurement_tstamp'].tolist(),
        'speed':form['speed'].tolist()
        }
reading_dict_selected = reading_dict[list(reading_dict.keys())[0]]
delta_reading_time = divmod((pd.to_datetime(reading_dict_selected['measurement_tstamp'][1], format='%Y-%m-%d %H:%M:%S') - pd.to_datetime(reading_dict_selected['measurement_tstamp'][0], format='%Y-%m-%d %H:%M:%S')).seconds,60)[0]




In [13]:
'''build measurement_tmc.csv''' 

measurement_tmc = pd.DataFrame()
measurement_tmc['link_id_tmc'] = None
measurement_tmc['corridor_id'] = None
measurement_tmc['corridor_link_order'] = None
measurement_tmc['from_node_id'] = None
measurement_tmc['to_node_id'] = None
measurement_tmc['time_period'] = None
measurement_tmc['date'] = None
measurement_tmc['geometry'] = None
measurement_tmc['volume'] = None
measurement_tmc['travel_time'] = None
measurement_tmc['speed'] = None
measurement_tmc['reference_speed'] = None
measurement_tmc['density'] = None
measurement_tmc['queue'] = None
measurement_tmc['notes'] = None


k=0
p=1
measurement_tmc_dict = {}
for i in link_tmc.index:
    try:
        reading_dict_selected = reading_dict[link_tmc['name'][i]]
        for j in range(0,len(reading_dict_selected['measurement_tstamp'])):
            speed = round(np.mean(reading_dict_selected['speed'][j:j+2]))
            volume_rate = round(volume_speed_func(speed))
            volume = round(volume_rate/60*delta_reading_time)
            density = round(volume_rate/speed)
            measurement_tmc_dict[k] = {'link_id_tmc': i,\
                                            'corridor_id': link_tmc['corridor_id'][i],\
                                            'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                            'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                            'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                            'time_period': reading_dict_selected['measurement_tstamp'][j][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j][14:16]+'_'+\
                                                reading_dict_selected['measurement_tstamp'][j+1][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j+1][14:16],\
                                            'date': reading_dict_selected['measurement_tstamp'][j][:10],\
                                            'geometry': link_tmc['geometry'][i],\
                                            'volume': volume,\
                                            'volume_rate': volume_rate,\
                                            'travel_time': None,\
                                            'speed':speed,\
                                            'reference_speed': None,\
                                            'density': density,\
                                            'queue': None,\
                                            'notes': None }
            k += 1

    except:
        measurement_tmc_dict[k] = {'link_id_tmc': i,\
                                        'corridor_id': link_tmc['corridor_id'][i],\
                                        'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                        'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                        'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                        'time_period': None,\
                                        'date': None,\
                                        'geometry': link_tmc['geometry'][i],\
                                        'volume': None,\
                                        'volume_rate': volume_rate,\
                                        'travel_time': None,\
                                        'speed': None,\
                                        'reference_speed': None,\
                                        'density': None,\
                                        'queue': None,\
                                        'notes': None }
        k += 1

    if link_tmc.index.get_loc(i) > p/10 * len(link_tmc): 
        print(str(p*10)+"%"+' measurement_tmc completed!')
        p = p + 1

measurement_tmc = pd.DataFrame(measurement_tmc_dict).transpose()
measurement_tmc = measurement_tmc.dropna(subset=['time_period']) #remove na at the end of day of unrecorded ones
measurement_tmc = measurement_tmc.reset_index()
measurement_tmc = measurement_tmc.drop(['index'], 1)
measurement_tmc.to_csv('measurement_tmc.csv',index = False)
print('measurement_tmc.csv generated!')

10% measurement_tmc completed!
20% measurement_tmc completed!
30% measurement_tmc completed!
40% measurement_tmc completed!
50% measurement_tmc completed!
60% measurement_tmc completed!
70% measurement_tmc completed!
80% measurement_tmc completed!
90% measurement_tmc completed!
measurement_tmc.csv generated!


In [14]:
measurement_tmc

,link_id_tmc,corridor_id,corridor_link_order,from_node_id,to_node_id,time_period,date,geometry,volume,volume_rate,travel_time,speed,reference_speed,density,queue,notes
0,100000001,VA-7_EASTBOUND,38,100000001,100000002,0000_0005,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",148,1779,None,44,None,40,None,None
1,100000001,VA-7_EASTBOUND,38,100000001,100000002,0005_0010,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",148,1779,None,44,None,40,None,None
2,100000001,VA-7_EASTBOUND,38,100000001,100000002,0010_0015,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",148,1779,None,44,None,40,None,None
3,100000001,VA-7_EASTBOUND,38,100000001,100000002,0015_0020,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",148,1779,None,44,None,40,None,None
4,100000001,VA-7_EASTBOUND,38,100000001,100000002,0020_0025,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",148,1779,None,44,None,40,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4937,100000061,VA-7_WESTBOUND,48,100000062,100000063,0630_0635,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",153,1833,None,52,None,35,None,None
4938,100000061,VA-7_WESTBOUND,48,100000062,100000063,0635_0640,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",148,1772,None,59,None,30,None,None
4939,100000061,VA-7_WESTBOUND,48,100000062,100000063,0640_0645,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",148,1772,None,59,None,30,None,None
4940,100000061,VA-7_WESTBOUND,48,100000062,100000063,0645_0650,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",153,1833,None,53,None,35,None,None


## step 3 mapping osm network to tmc network

In [15]:
link_base = pd.read_csv('link.csv', low_memory=False)
link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk'])]
link_base = link_base[-link_base['name'].isna()]
# link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk','primary','secondary'])]
link_base = link_base.reset_index()
link_base = link_base.drop(['index'], 1)

In [16]:
link_base

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway
0,Dulles Toll Road,855,8798447,10039,520,1,169.188317,4,55,NaN,motorway,1,"LINESTRING (-77.31180689999999 38.9475173, -77...",auto,0
1,Dulles Toll Road,856,8798447,520,435,1,427.643073,4,55,NaN,motorway,1,"LINESTRING (-77.309876 38.9477546, -77.3091781...",auto,0
2,Dulles Toll Road,938,8798925,438,11065,1,534.735507,4,55,NaN,motorway,1,"LINESTRING (-77.2639229 38.9334748, -77.263784...",auto,0
3,Dulles Toll Road,1040,8799968,727,729,1,953.242650,4,55,NaN,motorway,1,"LINESTRING (-77.33145740000001 38.9454158, -77...",auto,0
4,Capital Beltway,1049,8800050,9633,8696,1,190.021290,5,55,NaN,motorway,1,"LINESTRING (-77.2224422 38.8826937, -77.222107...",auto,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,Leesburg Pike,31112,936341159,2434,11555,1,42.649113,5,45,NaN,trunk,2,"LINESTRING (-77.2228317 38.9133617, -77.223227...",auto,0
888,495 Express Lanes,31113,939967934,14356,11680,1,88.168123,3,65,NaN,motorway,1,"LINESTRING (-77.2175216 38.9153263, -77.217492...",auto,0
889,Dulles Toll Road,31114,939970508,847,846,1,101.323438,4,55,NaN,motorway,1,"LINESTRING (-77.2435752 38.9344506, -77.243106...",auto,0
890,Dulles Toll Road,31115,939970508,846,845,1,146.592700,4,55,NaN,motorway,1,"LINESTRING (-77.2424048 38.9344861, -77.241861...",auto,0


In [17]:
def LLs2Dist(lon1, lat1, lon2, lat2): #WGS84 transfer coordinate system to distance(meter) #xy #credit to xtHuang0927
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c * 1000
    return distance

In [18]:
def getDegree(latA, lonA, latB, lonB):
    """
    Args:
        point p1(latA, lonA)
        point p2(latB, lonB)
    Returns:
        bearing between the two GPS points,
        default: the basis of heading direction is north
    """
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
    dLon = radLonB - radLonA
    y = math.sin(dLon) * math.cos(radLatB)
    x = math.cos(radLatA) * math.sin(radLatB) - math.sin(radLatA) * math.cos(radLatB) * math.cos(dLon)
    brng = np.degrees(math.atan2(y, x))
    brng = (brng + 360) % 360
    # brng = 360 - brng
    return brng
    

In [39]:
link_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,,
100000001,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,1,None,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",1,0.038620,None,None,None,None,None
100000002,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,1,None,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",1,0.164895,None,None,None,None,None
100000003,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,1,None,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",1,0.512158,None,None,None,None,None
100000004,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,1,None,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",1,0.682589,None,None,None,None,None
100000005,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,1,None,"LINESTRING (-77.3502 39.0028,-77.3455 39.0005)",1,0.296731,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000058,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,1,None,"LINESTRING (-77.346 39.001,-77.35 39.003)",1,0.254799,None,None,None,None,None
100000059,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,1,None,"LINESTRING (-77.35 39.003,-77.3609 39.009)",1,0.718593,None,None,None,None,None
100000060,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,1,None,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",1,0.421999,None,None,None,None,None


In [40]:
multiline_string_tmc_list = []
multiline_string_tmc_list_sub = []
for j in link_tmc.index:
    link_tmc_geometry_list = link_tmc.loc[j,'geometry'][12:-1].split(",")
    for link_tmc_geometry in link_tmc_geometry_list:
        multiline_string_tmc_list_sub.append((float(link_tmc_geometry.split(" ")[0]),float(link_tmc_geometry.split(" ")[1])))
    multiline_string_tmc_list_sub = tuple(multiline_string_tmc_list_sub)
    multiline_string_tmc_list.append(multiline_string_tmc_list_sub)
    multiline_string_tmc_list_sub = []

multiline_string_base_list = []
multiline_string_base_list_sub = []
for j in link_base.index:
    link_base_geometry_list = link_base.loc[j,'geometry'][12:-1].split(", ")
    for link_base_geometry in link_base_geometry_list:
        multiline_string_base_list_sub.append((float(link_base_geometry.split(" ")[0]),float(link_base_geometry.split(" ")[1])))
    multiline_string_base_list_sub = tuple(multiline_string_base_list_sub)
    multiline_string_base_list.append(multiline_string_base_list_sub)
    multiline_string_base_list_sub = []

from shapely.geometry import MultiLineString

line_tmc = MultiLineString(multiline_string_tmc_list)
line_base = MultiLineString(multiline_string_base_list) 
#bbox = left,bottom,right,top = min Longitude , min Latitude , max Longitude , max Latitude 

if (line_tmc.bounds[0] > line_base.bounds[0]) & (line_tmc.bounds[1] > line_base.bounds[1]) \
    & (line_tmc.bounds[2] < line_base.bounds[2]) & (line_tmc.bounds[3] < line_base.bounds[3]):
    matching_tmc2gmns_dict = {}
    k = 0
    p = 1

    for j in link_tmc.index:
        lon_tmc_list = []
        lat_tmc_list = []
        link_tmc_geometry_list = link_tmc.loc[j,'geometry'][12:-1].split(",")
        for link_tmc_geometry in link_tmc_geometry_list:
            lon_tmc_list.append(float(link_tmc_geometry.split(" ")[0]))
            lat_tmc_list.append(float(link_tmc_geometry.split(" ")[1]))
        center_tmc_lon = np.mean(lon_tmc_list)
        center_tmc_lat = np.mean(lat_tmc_list)
        tmc_lon_1 = lon_tmc_list[0]
        tmc_lon_2 = lon_tmc_list[-1]
        tmc_lat_1 = lat_tmc_list[0]
        tmc_lat_2 = lat_tmc_list[-1]
        if getDegree(tmc_lat_1,tmc_lon_1,tmc_lat_2,tmc_lon_2)>180:
            angle_tmc = getDegree(tmc_lat_2,tmc_lon_2,tmc_lat_1,tmc_lon_1)
        else:
            angle_tmc = getDegree(tmc_lat_1,tmc_lon_1,tmc_lat_2,tmc_lon_2)

        for dividing_rate in range(1,11):
            end_point_tmc = LineString([(tmc_lon_1, tmc_lat_1),  (tmc_lon_2, tmc_lat_2)]).interpolate(dividing_rate/10, normalized=True)
            start_point_tmc = LineString([(tmc_lon_1, tmc_lat_1),  (tmc_lon_2, tmc_lat_2)]).interpolate((dividing_rate-1)/10, normalized=True)

            distance_list = []
            angle_list = []
            for i in range(len(link_base)):
                lon_list = []
                lat_list = [] 
                point_base_list = []
                link_geometry_list = link_base.loc[i,'geometry'][12:-1].split(", ")
                for link_geometry in link_geometry_list:
                    lon_list.append(float(link_geometry.split(" ")[0]))
                    lat_list.append(float(link_geometry.split(" ")[1]))
                    point_base_list.append(tuple([float(link_geometry.split(" ")[0]),float(link_geometry.split(" ")[1])]))
                base_line = geom.LineString(tuple(point_base_list))
                '''distance'''
                distance_list.append(start_point_tmc.distance(base_line))
                
                '''angle '''
                base_lon_1 = lon_list[0]
                base_lon_2 = lon_list[-1]
                base_lat_1 = lat_list[0]
                base_lat_2 = lat_list[-1]
                if getDegree(base_lat_1,base_lon_1,base_lat_2,base_lon_2)>180:
                    angle_base = getDegree(base_lat_2,base_lon_2,base_lat_1,base_lon_1)
                else:
                    angle_base = getDegree(base_lat_1,base_lon_1,base_lat_2,base_lon_2)
                

                if abs(angle_tmc - angle_base) >= 90:
                    relative_angle = 180 - abs(angle_tmc - angle_base)
                else:
                    relative_angle = abs(angle_tmc - angle_base)
                angle_list.append(relative_angle)

            small_angle_list = [i for i, value in enumerate(angle_list) if value < 45]
            df_distance = pd.DataFrame({'distance':distance_list})
            
            nearest_index = df_distance.loc[small_angle_list].idxmin().values[0]

            matching_tmc2gmns_dict[k] = {'name_tmc':link_tmc.loc[j]['name'],\
                                        'corridor_id_tmc':link_tmc.loc[j]['corridor_id'],\
                                        'link_id_tmc':link_tmc.loc[[j]].index.values[0],\
                                        'from_node_id_tmc':link_tmc.loc[j]['from_node_id'],\
                                        'to_node_id_tmc':link_tmc.loc[j]['to_node_id'],\
                                        'category_id_tmc':link_tmc.index.get_loc(j)+1,\
                                        'geometry_tmc':link_tmc.loc[j]['geometry'],\
                                        'name_base':link_base['name'][nearest_index],\
                                        'link_id_base':link_base['link_id'][nearest_index],\
                                        'from_node_id_base':link_base['from_node_id'][nearest_index],\
                                        'to_node_id_base':link_base['to_node_id'][nearest_index],\
                                        'category_id_base':link_tmc.index.get_loc(j)+1,\
                                        'geometry_base':link_base['geometry'][nearest_index],\
                                        'distance':min(distance_list),\
                                        'geometry_tmc_base':'MULTILINESTRING ('+ link_tmc.loc[j]['geometry'][11:] + \
                                                            ', ' + link_base['geometry'][nearest_index][11:]+')'}
            k += 1

        
        if link_tmc.index.get_loc(j) > p/10 * len(link_tmc): 
            print(str(p*10)+"%"+' matching completed!')
            p = p + 1
            

    matching_tmc2gmns = pd.DataFrame(matching_tmc2gmns_dict).transpose()
    matching_tmc2gmns_drop_duplicates = matching_tmc2gmns.drop(columns=['distance']).drop_duplicates()
    matching_tmc2gmns_drop_duplicates['distance'] = matching_tmc2gmns.loc[matching_tmc2gmns_drop_duplicates.index]['distance']
    matching_tmc2gmns_drop_duplicates = matching_tmc2gmns_drop_duplicates.reset_index()
    matching_tmc2gmns_drop_duplicates = matching_tmc2gmns_drop_duplicates.drop(['index'], 1)
    matching_tmc2gmns = matching_tmc2gmns_drop_duplicates

    matching_tmc2gmns.to_csv('matching_tmc2gmns.csv',index = False)
    print('matching_tmc2gmns.csv generated!')
else:
    print('base map cannot cover all TMC links, please use larger base map')

10% matching completed!
20% matching completed!
30% matching completed!
40% matching completed!
50% matching completed!
60% matching completed!
70% matching completed!
80% matching completed!
90% matching completed!
matching_tmc2gmns.csv generated!


In [41]:
matching_name_base_counts = matching_tmc2gmns['name_base'].value_counts()
link_base = link_base[link_base['name'].isin(matching_name_base_counts[matching_name_base_counts>10].index)]
link_base = link_base.reset_index()
link_base = link_base.drop(['index'], 1)
link_base.to_csv('link_base_small.csv',index = False)

In [42]:
link_base

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway
0,Leesburg Pike,3097,8810199,10992,12628,1,15.134541,2,35,NaN,trunk,2,"LINESTRING (-77.19028350000001 38.8946975, -77...",auto,0
1,Leesburg Pike,3098,8810199,12628,10994,1,162.845494,2,35,NaN,trunk,2,"LINESTRING (-77.1904157 38.8947866, -77.190474...",auto,0
2,Leesburg Pike,18484,38166809,2481,9736,1,154.705356,6,35,NaN,trunk,2,"LINESTRING (-77.21855739999999 38.9108106, -77...",auto,0
3,Leesburg Pike,18485,38166812,9736,13902,1,58.600278,6,35,NaN,trunk,2,"LINESTRING (-77.21708959999999 38.9100161, -77...",auto,0
4,Leesburg Pike,18488,38168243,1894,2494,1,131.305456,4,35,NaN,trunk,2,"LINESTRING (-77.2472387 38.9346714, -77.246315...",auto,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Leesburg Pike,30984,911652845,11555,11554,1,52.054544,4,45,NaN,trunk,2,"LINESTRING (-77.2232279 38.9135899, -77.223406...",auto,0
305,Leesburg Pike,31005,917681735,11107,2486,1,53.243449,5,35,NaN,trunk,2,"LINESTRING (-77.2270954 38.915796, -77.2267063...",auto,0
306,Leesburg Pike,31110,936232557,14355,1085,1,22.528303,5,35,NaN,trunk,2,"LINESTRING (-77.2154348 38.9090859, -77.215223...",auto,0
307,Leesburg Pike,31111,936341159,2433,2434,1,103.519204,5,45,NaN,trunk,2,"LINESTRING (-77.2218574 38.9128217, -77.222224...",auto,0


In [43]:
# extract base route
link_base_iteration = link_base 
base_route_dict = {}

i = 0
while len(link_base_iteration)>0:
    k = link_base_iteration.index[0]
    base_route_index_list = []
    base_route_list = []
    base_route_index_list.append(k)
    base_route_list.append(link_base.loc[k,'from_node_id'])
    base_route_list.append(link_base.loc[k,'to_node_id'])
    link_base_iteration = link_base_iteration.drop(k)

    selected_link_base_row = link_base_iteration[link_base_iteration['from_node_id'] == base_route_list[-1]]
    while len(selected_link_base_row) != 0:
        base_route_list.append(selected_link_base_row['to_node_id'].values[0])
        base_route_index_list.append(selected_link_base_row.index.values[0])
        link_base_iteration = link_base_iteration.drop(selected_link_base_row.index.values[0])
        selected_link_base_row = link_base_iteration[link_base_iteration['from_node_id'] == base_route_list[-1]]

    selected_link_base_row = link_base_iteration[link_base_iteration['to_node_id'] == base_route_list[0]]
    while len(selected_link_base_row) != 0:
        base_route_list.insert(0,selected_link_base_row['from_node_id'].values[0])
        base_route_index_list.insert(0,selected_link_base_row.index.values[0])
        link_base_iteration = link_base_iteration.drop(selected_link_base_row.index.values[0])
        selected_link_base_row = link_base_iteration[link_base_iteration['to_node_id'] == base_route_list[0]]

    base_route_dict[i] = {
        'base_route_list':base_route_list,
        'base_route_index_list':base_route_index_list
    }
    i+=1

In [44]:
link_base_iteration

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway


In [45]:
# extract tmc route
tmc_route_dict = {}
gp = matching_tmc2gmns.groupby('corridor_id_tmc')
for key, form in gp:
    tmc_route_dict[key] = {
        'from_node_id_base':set(form['from_node_id_base'].tolist()),
        'tmc_index':list(form.index.values)
        }


base_route_tmc_dict = {}
k=0
for i in range(len(base_route_dict)):
    for key, value in tmc_route_dict.items():
        base_route_tmc_dict[k] = {'base_route':i,
        'corridor': key,
        'tmc_index':value['tmc_index'],
        'number': int(len(set(base_route_dict[i]['base_route_list']).intersection(value['from_node_id_base'])))}
        k+=1

base_route_tmc = pd.DataFrame(base_route_tmc_dict).transpose()      
base_route_tmc

,base_route,corridor,tmc_index,number
0,0,VA-7_EASTBOUND,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",17
1,0,VA-7_WESTBOUND,"[174, 175, 176, 177, 178, 179, 180, 181, 182, ...",78
2,1,VA-7_EASTBOUND,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",129
3,1,VA-7_WESTBOUND,"[174, 175, 176, 177, 178, 179, 180, 181, 182, ...",43
4,2,VA-7_EASTBOUND,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",4
5,2,VA-7_WESTBOUND,"[174, 175, 176, 177, 178, 179, 180, 181, 182, ...",31


In [51]:
base_tmc_dict = {}
gp = base_route_tmc.groupby('base_route')
for key, form in gp:
    base_tmc_dict[key] = {
        'corridor':form.loc[pd.to_numeric(form['number']).idxmax()]['corridor']
        }
base_tmc = pd.DataFrame(base_tmc_dict).transpose() 
base_tmc

,corridor
0,VA-7_WESTBOUND
1,VA-7_EASTBOUND
2,VA-7_WESTBOUND


In [47]:
tmc_base_dict = {}
gp = base_tmc.groupby('corridor')
for key, form in gp:
    tmc_base_dict[key] = list(form.index.values)
tmc_base_dict

{'VA-7_EASTBOUND': [1], 'VA-7_WESTBOUND': [0, 2]}

In [48]:

matching_tmc2gmns_dict = {}
k = 0
p = 1
tmc_count = 0
for key, value in tmc_base_dict.items():
    link_tmc_sub = link_tmc[link_tmc['corridor_id'] == key]
    base_link_list_sub = []
    for base_route_id in value:
        base_link_list_sub += base_route_dict[base_route_id]['base_route_index_list']
    link_base_sub = link_base.loc[base_link_list_sub]


    for j in link_tmc_sub.index:
        lon_tmc_list = []
        lat_tmc_list = []
        link_tmc_geometry_list = link_tmc_sub.loc[j,'geometry'][12:-1].split(",")
        for link_tmc_geometry in link_tmc_geometry_list:
            lon_tmc_list.append(float(link_tmc_geometry.split(" ")[0]))
            lat_tmc_list.append(float(link_tmc_geometry.split(" ")[1]))
        center_tmc_lon = np.mean(lon_tmc_list)
        center_tmc_lat = np.mean(lat_tmc_list)
        tmc_lon_1 = lon_tmc_list[0]
        tmc_lon_2 = lon_tmc_list[-1]
        tmc_lat_1 = lat_tmc_list[0]
        tmc_lat_2 = lat_tmc_list[-1]
        if getDegree(tmc_lat_1,tmc_lon_1,tmc_lat_2,tmc_lon_2)>180:
            angle_tmc = getDegree(tmc_lat_2,tmc_lon_2,tmc_lat_1,tmc_lon_1)
        else:
            angle_tmc = getDegree(tmc_lat_1,tmc_lon_1,tmc_lat_2,tmc_lon_2)

        for dividing_rate in range(1,51):
            end_point_tmc = LineString([(tmc_lon_1, tmc_lat_1),  (tmc_lon_2, tmc_lat_2)]).interpolate(dividing_rate/50, normalized=True)
            start_point_tmc = LineString([(tmc_lon_1, tmc_lat_1),  (tmc_lon_2, tmc_lat_2)]).interpolate((dividing_rate-1)/50, normalized=True)

            distance_list = []
            angle_list = []
            for i in link_base_sub.index:
                lon_list = []
                lat_list = [] 
                point_base_list = []
                link_geometry_list = link_base_sub.loc[i,'geometry'][12:-1].split(", ")
                for link_geometry in link_geometry_list:
                    lon_list.append(float(link_geometry.split(" ")[0]))
                    lat_list.append(float(link_geometry.split(" ")[1]))
                    point_base_list.append(tuple([float(link_geometry.split(" ")[0]),float(link_geometry.split(" ")[1])]))
                base_line = geom.LineString(tuple(point_base_list))
                '''distance'''
                distance_list.append(start_point_tmc.distance(base_line))
                
                '''angle '''
                base_lon_1 = lon_list[0]
                base_lon_2 = lon_list[-1]
                base_lat_1 = lat_list[0]
                base_lat_2 = lat_list[-1]
                if getDegree(base_lat_1,base_lon_1,base_lat_2,base_lon_2)>180:
                    angle_base = getDegree(base_lat_2,base_lon_2,base_lat_1,base_lon_1)
                else:
                    angle_base = getDegree(base_lat_1,base_lon_1,base_lat_2,base_lon_2)
                

                if abs(angle_tmc - angle_base) >= 90:
                    relative_angle = 180 - abs(angle_tmc - angle_base)
                else:
                    relative_angle = abs(angle_tmc - angle_base)
                angle_list.append(relative_angle)

            small_angle_list = [i for i, value in enumerate(angle_list) if value < 45]
            df_distance = pd.DataFrame({'distance':distance_list})
            
            nearest_index = df_distance.loc[small_angle_list].idxmin().values[0]

            matching_tmc2gmns_dict[k] = {'name_tmc':link_tmc_sub.loc[j]['name'],\
                                        'corridor_id_tmc':link_tmc_sub.loc[j]['corridor_id'],\
                                        'link_id_tmc':link_tmc_sub.loc[[j]].index.values[0],\
                                        'from_node_id_tmc':link_tmc_sub.loc[j]['from_node_id'],\
                                        'to_node_id_tmc':link_tmc_sub.loc[j]['to_node_id'],\
                                        'category_id_tmc':link_tmc_sub.index.get_loc(j)+1,\
                                        'geometry_tmc':link_tmc_sub.loc[j]['geometry'],\
                                        'name_base':link_base_sub.iloc[nearest_index]['name'],\
                                        'link_id_base':link_base_sub.iloc[nearest_index]['link_id'],\
                                        'from_node_id_base':link_base_sub.iloc[nearest_index]['from_node_id'],\
                                        'to_node_id_base':link_base_sub.iloc[nearest_index]['to_node_id'],\
                                        'category_id_base':link_tmc_sub.index.get_loc(j)+1,\
                                        'geometry_base':link_base_sub.iloc[nearest_index]['geometry'],\
                                        'distance':min(distance_list),\
                                        'geometry_tmc_base':'MULTILINESTRING ('+ link_tmc_sub.loc[j]['geometry'][11:] + \
                                                            ', ' + link_base_sub.iloc[nearest_index]['geometry'][11:]+')'}
            k += 1

        tmc_count += 1
        if tmc_count > p/10 * len(link_tmc): 
            print(str(p*10)+"%"+' matching completed!')
            p = p + 1
        

matching_tmc2gmns = pd.DataFrame(matching_tmc2gmns_dict).transpose()
matching_tmc2gmns_drop_duplicates = matching_tmc2gmns.drop(columns=['distance']).drop_duplicates()
matching_tmc2gmns_drop_duplicates['distance'] = matching_tmc2gmns.loc[matching_tmc2gmns_drop_duplicates.index]['distance']
matching_tmc2gmns_drop_duplicates = matching_tmc2gmns_drop_duplicates.reset_index()
matching_tmc2gmns_drop_duplicates = matching_tmc2gmns_drop_duplicates.drop(['index'], 1)
matching_tmc2gmns = matching_tmc2gmns_drop_duplicates

matching_tmc2gmns.to_csv('matching_tmc2gmns.csv',index = False)
print('matching_tmc2gmns.csv generated!')


10% matching completed!
20% matching completed!
30% matching completed!
40% matching completed!
50% matching completed!
60% matching completed!
70% matching completed!
80% matching completed!
90% matching completed!
matching_tmc2gmns.csv generated!


In [49]:
set(matching_tmc2gmns['name_base'])

{'Leesburg Pike'}

In [50]:
matching_tmc2gmns.head(20)

,name_tmc,corridor_id_tmc,link_id_tmc,from_node_id_tmc,to_node_id_tmc,category_id_tmc,geometry_tmc,name_base,link_id_base,from_node_id_base,to_node_id_base,category_id_base,geometry_base,geometry_tmc_base,distance
0,110N05711,VA-7_EASTBOUND,100000001,100000001,100000002,1,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",Leesburg Pike,28048,13131,9641,1,"LINESTRING (-77.3710543 39.0141705, -77.370912...","MULTILINESTRING ((-77.3715 39.0144,-77.3709 39...",0.000501317
1,110-05710,VA-7_EASTBOUND,100000002,100000002,100000003,2,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",Leesburg Pike,18653,9641,9642,2,"LINESTRING (-77.37091289999999 39.0140956, -77...","MULTILINESTRING ((-77.3709 39.0141,-77.3684 39...",9.83139e-06
2,110-05710,VA-7_EASTBOUND,100000002,100000002,100000003,2,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",Leesburg Pike,19078,9642,9643,2,"LINESTRING (-77.36887400000001 39.0130269, -77...","MULTILINESTRING ((-77.3709 39.0141,-77.3684 39...",5.61382e-05
3,110-05710,VA-7_EASTBOUND,100000002,100000002,100000003,2,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",Leesburg Pike,29356,9643,9644,2,"LINESTRING (-77.368475 39.0128263, -77.3679666...","MULTILINESTRING ((-77.3709 39.0141,-77.3684 39...",7.40194e-05
4,110N05710,VA-7_EASTBOUND,100000003,100000003,100000004,3,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",Leesburg Pike,29356,9643,9644,3,"LINESTRING (-77.368475 39.0128263, -77.3679666...","MULTILINESTRING ((-77.3684 39.0127,-77.3605 39...",7.89006e-05
5,110N05710,VA-7_EASTBOUND,100000003,100000003,100000004,3,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",Leesburg Pike,19075,9644,12392,3,"LINESTRING (-77.36452420000001 39.0108146, -77...","MULTILINESTRING ((-77.3684 39.0127,-77.3605 39...",0.000109043
6,110N05710,VA-7_EASTBOUND,100000003,100000003,100000004,3,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",Leesburg Pike,19076,12392,9992,3,"LINESTRING (-77.3606994 39.0086931, -77.360374...","MULTILINESTRING ((-77.3684 39.0127,-77.3605 39...",1.04079e-05
7,110-05709,VA-7_EASTBOUND,100000004,100000004,100000005,4,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",Leesburg Pike,19076,12392,9992,4,"LINESTRING (-77.3606994 39.0086931, -77.360374...","MULTILINESTRING ((-77.3605 39.0086,-77.3502 39...",1.54818e-05
8,110-05709,VA-7_EASTBOUND,100000004,100000004,100000005,4,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",Leesburg Pike,19890,9992,13666,4,"LINESTRING (-77.3579458 39.0071639, -77.357303...","MULTILINESTRING ((-77.3605 39.0086,-77.3502 39...",4.38142e-06
9,110-05709,VA-7_EASTBOUND,100000004,100000004,100000005,4,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",Leesburg Pike,19891,13666,9993,4,"LINESTRING (-77.3553556 39.0057424, -77.354373...","MULTILINESTRING ((-77.3605 39.0086,-77.3502 39...",3.44037e-05


In [37]:

matching_tmc2gmns_dict = {}
k = 0
p = 1

for j in link_tmc.index:
    lon_tmc_list = []
    lat_tmc_list = []
    link_tmc_geometry_list = link_tmc.loc[j,'geometry'][12:-1].split(",")
    for link_tmc_geometry in link_tmc_geometry_list:
        lon_tmc_list.append(float(link_tmc_geometry.split(" ")[0]))
        lat_tmc_list.append(float(link_tmc_geometry.split(" ")[1]))
    center_tmc_lon = np.mean(lon_tmc_list)
    center_tmc_lat = np.mean(lat_tmc_list)
    tmc_lon_1 = lon_tmc_list[0]
    tmc_lon_2 = lon_tmc_list[-1]
    tmc_lat_1 = lat_tmc_list[0]
    tmc_lat_2 = lat_tmc_list[-1]
    if getDegree(tmc_lat_1,tmc_lon_1,tmc_lat_2,tmc_lon_2)>180:
        angle_tmc = getDegree(tmc_lat_2,tmc_lon_2,tmc_lat_1,tmc_lon_1)
    else:
        angle_tmc = getDegree(tmc_lat_1,tmc_lon_1,tmc_lat_2,tmc_lon_2)

    for dividing_rate in range(1,11):
        end_point_tmc = LineString([(tmc_lon_1, tmc_lat_1),  (tmc_lon_2, tmc_lat_2)]).interpolate(dividing_rate/10, normalized=True)
        start_point_tmc = LineString([(tmc_lon_1, tmc_lat_1),  (tmc_lon_2, tmc_lat_2)]).interpolate((dividing_rate-1)/10, normalized=True)

        distance_list = []
        angle_list = []
        for i in range(len(link_base)):
            lon_list = []
            lat_list = [] 
            point_base_list = []
            link_geometry_list = link_base.loc[i,'geometry'][12:-1].split(", ")
            for link_geometry in link_geometry_list:
                lon_list.append(float(link_geometry.split(" ")[0]))
                lat_list.append(float(link_geometry.split(" ")[1]))
                point_base_list.append(tuple([float(link_geometry.split(" ")[0]),float(link_geometry.split(" ")[1])]))
            base_line = geom.LineString(tuple(point_base_list))
            '''distance'''
            distance_list.append(start_point_tmc.distance(base_line))
            
            '''angle '''
            base_lon_1 = lon_list[0]
            base_lon_2 = lon_list[-1]
            base_lat_1 = lat_list[0]
            base_lat_2 = lat_list[-1]
            if getDegree(base_lat_1,base_lon_1,base_lat_2,base_lon_2)>180:
                angle_base = getDegree(base_lat_2,base_lon_2,base_lat_1,base_lon_1)
            else:
                angle_base = getDegree(base_lat_1,base_lon_1,base_lat_2,base_lon_2)
            

            if abs(angle_tmc - angle_base) >= 90:
                relative_angle = 180 - abs(angle_tmc - angle_base)
            else:
                relative_angle = abs(angle_tmc - angle_base)
            angle_list.append(relative_angle)

        small_angle_list = [i for i, value in enumerate(angle_list) if value < 45]
        df_distance = pd.DataFrame({'distance':distance_list})
        
        nearest_index = df_distance.loc[small_angle_list].idxmin().values[0]

        matching_tmc2gmns_dict[k] = {'name_tmc':link_tmc.loc[j]['name'],\
                                    'corridor_id_tmc':link_tmc.loc[j]['corridor_id'],\
                                    'link_id_tmc':link_tmc.loc[[j]].index.values[0],\
                                    'from_node_id_tmc':link_tmc.loc[j]['from_node_id'],\
                                    'to_node_id_tmc':link_tmc.loc[j]['to_node_id'],\
                                    'category_id_tmc':link_tmc.index.get_loc(j)+1,\
                                    'geometry_tmc':link_tmc.loc[j]['geometry'],\
                                    'name_base':link_base['name'][nearest_index],\
                                    'link_id_base':link_base['link_id'][nearest_index],\
                                    'from_node_id_base':link_base['from_node_id'][nearest_index],\
                                    'to_node_id_base':link_base['to_node_id'][nearest_index],\
                                    'category_id_base':link_tmc.index.get_loc(j)+1,\
                                    'geometry_base':link_base['geometry'][nearest_index],\
                                    'distance':min(distance_list),\
                                    'geometry_tmc_base':'MULTILINESTRING ('+ link_tmc.loc[j]['geometry'][11:] + \
                                                        ', ' + link_base['geometry'][nearest_index][11:]+')'}
        k += 1

    
    if link_tmc.index.get_loc(j) > p/10 * len(link_tmc): 
        print(str(p*10)+"%"+' matching completed!')
        p = p + 1
        

matching_tmc2gmns = pd.DataFrame(matching_tmc2gmns_dict).transpose()
matching_tmc2gmns_drop_duplicates = matching_tmc2gmns.drop(columns=['distance']).drop_duplicates()
matching_tmc2gmns_drop_duplicates['distance'] = matching_tmc2gmns.loc[matching_tmc2gmns_drop_duplicates.index]['distance']
matching_tmc2gmns_drop_duplicates = matching_tmc2gmns_drop_duplicates.reset_index()
matching_tmc2gmns_drop_duplicates = matching_tmc2gmns_drop_duplicates.drop(['index'], 1)
matching_tmc2gmns = matching_tmc2gmns_drop_duplicates

matching_tmc2gmns.to_csv('matching_tmc2gmns.csv',index = False)
print('matching_tmc2gmns.csv generated!')


10% matching completed!
20% matching completed!
30% matching completed!
40% matching completed!
50% matching completed!
60% matching completed!
70% matching completed!
80% matching completed!
90% matching completed!
matching_tmc2gmns.csv generated!


In [86]:
matching_tmc2gmns

,name_tmc,corridor_id_tmc,link_id_tmc,from_node_id_tmc,to_node_id_tmc,category_id_tmc,geometry_tmc,name_base,link_id_base,from_node_id_base,to_node_id_base,category_id_base,geometry_base,geometry_tmc_base,distance
0,110N05711,VA-7_EASTBOUND,100000001,100000001,100000002,1,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",Leesburg Pike,18653,9641,9642,1,"LINESTRING (-77.37091289999999 39.0140956, -77...","MULTILINESTRING ((-77.3715 39.0144,-77.3709 39...",0.000661321
1,110-05710,VA-7_EASTBOUND,100000002,100000002,100000003,2,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",Leesburg Pike,18653,9641,9642,2,"LINESTRING (-77.37091289999999 39.0140956, -77...","MULTILINESTRING ((-77.3709 39.0141,-77.3684 39...",9.83139e-06
2,110N05710,VA-7_EASTBOUND,100000003,100000003,100000004,3,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",Leesburg Pike,18653,9641,9642,3,"LINESTRING (-77.37091289999999 39.0140956, -77...","MULTILINESTRING ((-77.3684 39.0127,-77.3605 39...",0.000575795
3,110N05710,VA-7_EASTBOUND,100000003,100000003,100000004,3,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",Leesburg Pike,19890,9992,13666,3,"LINESTRING (-77.3579458 39.0071639, -77.357303...","MULTILINESTRING ((-77.3684 39.0127,-77.3605 39...",0.00127717
4,110-05709,VA-7_EASTBOUND,100000004,100000004,100000005,4,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",Leesburg Pike,19890,9992,13666,4,"LINESTRING (-77.3579458 39.0071639, -77.357303...","MULTILINESTRING ((-77.3605 39.0086,-77.3502 39...",0.00293024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,110P05710,VA-7_WESTBOUND,100000060,100000061,100000062,29,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",Leesburg Pike,19075,9644,12392,29,"LINESTRING (-77.36452420000001 39.0108146, -77...","MULTILINESTRING ((-77.3609 39.009,-77.3673 39....",0.000171471
92,110P05710,VA-7_WESTBOUND,100000060,100000061,100000062,29,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",Leesburg Pike,19078,9642,9643,29,"LINESTRING (-77.36887400000001 39.0130269, -77...","MULTILINESTRING ((-77.3609 39.009,-77.3673 39....",0.00193691
93,110+05711,VA-7_WESTBOUND,100000061,100000062,100000063,30,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",Leesburg Pike,19078,9642,9643,30,"LINESTRING (-77.36887400000001 39.0130269, -77...","MULTILINESTRING ((-77.3673 39.0125,-77.3707 39...",0.00121947
94,110+05711,VA-7_WESTBOUND,100000061,100000062,100000063,30,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",Leesburg Pike,19077,10236,10237,30,"LINESTRING (-77.3684813 39.0131606, -77.368831...","MULTILINESTRING ((-77.3673 39.0125,-77.3707 39...",0.0002014


## step 4 generating measurement_osm 

In [37]:
link_base = pd.read_csv('link.csv', low_memory=False)
link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk'])]
link_base = link_base[-link_base['name'].isna()]
# link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk','primary','secondary'])]
link_base = link_base.reset_index()
link_base = link_base.drop(['index'], 1)

'''build measurement_base.csv''' 

matching_tmc2gmns_dict = {}
gp = matching_tmc2gmns.groupby('link_id_base')
for key, form in gp:
    matching_tmc2gmns_dict[key] = {
        'link_id_tmc':form['link_id_tmc'].tolist()
        }


k=0
p=1
i=0
measurement_base_dict = {}
for key, value in matching_tmc2gmns_dict.items():
    i += 1
    try:
        link_base_selected = link_base[link_base['link_id'] == key]
        measurement_tmc_selected = measurement_tmc[measurement_tmc['link_id_tmc'].isin(value['link_id_tmc'])]
        gp_measurement_tmc_selected = measurement_tmc_selected.groupby(['time_period','date'])
        for key_measurement_tmc_selected, form_measurement_tmc_selected in gp_measurement_tmc_selected:
            measurement_base_dict[k] = {'link_id': link_base_selected['link_id'].values[0],\
                                            'osm_way_id':link_base_selected['osm_way_id'].values[0],\
                                            'from_node_id': link_base_selected['from_node_id'].values[0],\
                                            'to_node_id': link_base_selected['to_node_id'].values[0],\
                                            'lanes': link_base_selected['lanes'].values[0], \
                                            'length': link_base_selected['length'].values[0], \
                                            'link_type_name': link_base_selected['link_type_name'].values[0], \
                                            'corridor_id': form_measurement_tmc_selected['corridor_id'].tolist()[0],\
                                            'time_period': key_measurement_tmc_selected[0],\
                                            'date': key_measurement_tmc_selected[1],\
                                            'geometry': link_base_selected['geometry'].values[0],\
                                            'volume': round(form_measurement_tmc_selected['volume'].mean()),\
                                            'volume_rate': round(form_measurement_tmc_selected['volume_rate'].mean()),\
                                            'speed': round(form_measurement_tmc_selected['speed'].mean()),\
                                            'density': round(form_measurement_tmc_selected['density'].mean()),\
                                            'ip_address': 'www.openstreetmap.org/?way=' + str(link_base_selected['osm_way_id'].values[0])} 

            k += 1
    except:
        measurement_base_dict[k] = {'link_id': link_base_selected['link_id'].values[0],\
                                            'osm_way_id':link_base_selected['osm_way_id'].values[0],\
                                            'from_node_id': link_base_selected['from_node_id'].values[0],\
                                            'to_node_id': link_base_selected['to_node_id'].values[0],\
                                            'lanes': link_base_selected['lanes'].values[0], \
                                            'length': link_base_selected['length'].values[0], \
                                            'link_type_name': link_base_selected['link_type_name'].values[0], \
                                            'corridor_id': None,\
                                            'time_period':None,\
                                            'date': None,\
                                            'geometry': link_base_selected['geometry'].values[0],\
                                            'volume': None,\
                                            'volume_rate': None,\
                                            'speed': None,\
                                            'density': None,\
                                            'ip_address': 'www.openstreetmap.org/?way=' + str(link_base_selected['osm_way_id'].values[0])}

        k += 1
    
    if i+1 > p/10 * len(matching_tmc2gmns_dict): 
        print(str(p*10)+"%"+' measurement_base completed!')
        p = p + 1

measurement_base = pd.DataFrame(measurement_base_dict).transpose()
measurement_base.to_csv('measurement_base.csv',index = False)
print('measurement_base.csv generated!')

10% measurement_base completed!
20% measurement_base completed!
30% measurement_base completed!
40% measurement_base completed!
50% measurement_base completed!
60% measurement_base completed!
70% measurement_base completed!
80% measurement_base completed!
90% measurement_base completed!
100% measurement_base completed!
measurement_base.csv generated!


In [38]:
measurement_base

,link_id,osm_way_id,from_node_id,to_node_id,lanes,length,link_type_name,corridor_id,time_period,date,geometry,volume,volume_rate,speed,density,ip_address
0,18484,38166809,2481,9736,6,154.705,trunk,VA-7_EASTBOUND,0000_0005,2015-04-01,"LINESTRING (-77.21855739999999 38.9108106, -77...",133,1597,33,48,www.openstreetmap.org/?way=38166809
1,18484,38166809,2481,9736,6,154.705,trunk,VA-7_EASTBOUND,0005_0010,2015-04-01,"LINESTRING (-77.21855739999999 38.9108106, -77...",133,1597,33,48,www.openstreetmap.org/?way=38166809
2,18484,38166809,2481,9736,6,154.705,trunk,VA-7_EASTBOUND,0010_0015,2015-04-01,"LINESTRING (-77.21855739999999 38.9108106, -77...",133,1597,33,48,www.openstreetmap.org/?way=38166809
3,18484,38166809,2481,9736,6,154.705,trunk,VA-7_EASTBOUND,0015_0020,2015-04-01,"LINESTRING (-77.21855739999999 38.9108106, -77...",135,1618,34,48,www.openstreetmap.org/?way=38166809
4,18484,38166809,2481,9736,6,154.705,trunk,VA-7_EASTBOUND,0020_0025,2015-04-01,"LINESTRING (-77.21855739999999 38.9108106, -77...",141,1692,38,45,www.openstreetmap.org/?way=38166809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19696,31111,936341159,2433,2434,5,103.519,trunk,VA-7_WESTBOUND,0635_0640,2015-04-01,"LINESTRING (-77.2218574 38.9128217, -77.222224...",140,1675,37,45,www.openstreetmap.org/?way=936341159
19697,31111,936341159,2433,2434,5,103.519,trunk,VA-7_WESTBOUND,0640_0645,2015-04-01,"LINESTRING (-77.2218574 38.9128217, -77.222224...",136,1638,35,47,www.openstreetmap.org/?way=936341159
19698,31111,936341159,2433,2434,5,103.519,trunk,VA-7_WESTBOUND,0645_0650,2015-04-01,"LINESTRING (-77.2218574 38.9128217, -77.222224...",131,1576,32,49,www.openstreetmap.org/?way=936341159
19699,31111,936341159,2433,2434,5,103.519,trunk,VA-7_WESTBOUND,0650_0655,2015-04-01,"LINESTRING (-77.2218574 38.9128217, -77.222224...",122,1460,27,54,www.openstreetmap.org/?way=936341159


In [26]:
print( __name__)
print(os.path.dirname(os.path.realpath('__file__')))
print(os.path.realpath('__file__'))
print(os.getcwd())

__main__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/__file__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
